In [ ]:
!pip install tensorflow keras numpy

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import glob

all_text = ""
for fname in sorted(glob.glob("dataset/*.txt")):
    with open(fname, encoding="utf-8") as f:
        all_text += f.read() + "\n"

print(f"Total corpus length: {len(all_text):,} characters")

In [ ]:
vocab = sorted(set(all_text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = {i: u for i, u in enumerate(vocab)}

print(f"Unique characters: {len(vocab)}")

In [ ]:
import numpy as np
from keras.utils import Sequence

class CharSequenceGenerator(Sequence):
    def __init__(self, text, seq_length, batch_size, vocab, step=1):
        self.text = text
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.vocab = vocab
        self.char2idx = {c: i for i, c in enumerate(vocab)}
        self.indices = list(range(0, len(text) - seq_length - 1, step))

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        x = np.zeros((self.batch_size, self.seq_length, len(self.vocab)), dtype=np.bool_)
        y = np.zeros((self.batch_size, len(self.vocab)), dtype=np.bool_)
        for i in range(self.batch_size):
            start = self.indices[idx * self.batch_size + i]
            seq = self.text[start:start + self.seq_length]
            next_char = self.text[start + self.seq_length]
            for t, char in enumerate(seq):
                x[i, t, self.char2idx[char]] = 1
            y[i, self.char2idx[next_char]] = 1
        return x, y

In [12]:
SEQ_LENGTH = 100
BATCH_SIZE = 256
generator = CharSequenceGenerator(all_text, SEQ_LENGTH, BATCH_SIZE, vocab, step=10)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(256, input_shape=(SEQ_LENGTH, len(vocab)), return_sequences=True),
    Dropout(0.2),
    LSTM(256),
    Dropout(0.2),
    Dense(len(vocab), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

In [15]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

EPOCHS = 20

callbacks = [
    ModelCheckpoint("lstm_textgen_best.h5", save_best_only=True),
    EarlyStopping(patience=3, restore_best_weights=True)
]

model.fit(generator, epochs=EPOCHS, callbacks=callbacks)

Epoch 1/20


/home/simon/Documents/python/autocorrect-linux/venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13137/13137 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 2.7228

/home/simon/Documents/python/autocorrect-linux/venv/lib/python3.10/site-packages/keras/src/callbacks/model_checkpoint.py:302: UserWarning: Can save best model only with val_loss available.
  if self._should_save_model(epoch, batch, logs, filepath):


13137/13137 ━━━━━━━━━━━━━━━━━━━━ 1748s 133ms/step - loss: 2.7228
Epoch 2/20


...

13137/13137 ━━━━━━━━━━━━━━━━━━━━ 1534s 117ms/step - loss: 1.3261
Epoch 17/20
13137/13137 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 1.3195

13137/13137 ━━━━━━━━━━━━━━━━━━━━ 1533s 117ms/step - loss: 1.3195
Epoch 18/20
13137/13137 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 1.3189

13137/13137 ━━━━━━━━━━━━━━━━━━━━ 1530s 116ms/step - loss: 1.3189
Epoch 19/20
13137/13137 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 1.3166

13137/13137 ━━━━━━━━━━━━━━━━━━━━ 1531s 117ms/step - loss: 1.3166
Epoch 20/20
13137/13137 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 1.3110

13137/13137 ━━━━━━━━━━━━━━━━━━━━ 1529s 116ms/step - loss: 1.3110
